<a href="https://colab.research.google.com/github/JParraArias/SignalsAndSystems2024-2/blob/main/Dashboard_2S2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PUNTO 2

El siguiente bloque de código instala varias bibliotecas necesarias para el proyecto:

- **`streamlit`**: Framework para crear aplicaciones web interactivas en Python.
- **`localtunnel`** (vía `npm`): Permite exponer un puerto local a la web.
- **`yt-dlp`**: Herramienta avanzada para descargar videos de YouTube y otras plataformas.
- **`pyngrok`**: Facilita la creación de túneles seguros con `ngrok`, útil para exponer servicios locales.
- **`sympy`**: Biblioteca para cálculos simbólicos en matemáticas.
- **`soundfile`**: Permite leer y escribir archivos de audio en distintos formatos.
- **`pytube`**: Otra biblioteca para descargar videos de YouTube.
- **`ffmpeg`**: Software para procesamiento y conversión de archivos de audio y video.
- **`pandas, gspread, oauth2client`**: Facilitan la manipulación de datos y la conexión con Google Sheets.
- **`numpy, matplotlib, scipy`**: Librerías científicas para análisis numérico y visualización.

Se reinstala `yt-dlp` desde su repositorio de GitHub para asegurar la versión más reciente.
Finalmente, se instalan todas las bibliotecas necesarias en una única línea para evitar posibles conflictos de dependencias.

In [13]:
!pip install streamlit -q
!npm install localtunnel
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install pyngrok
!pip install sympy --upgrade
!pip install yt-dlp
!pip install soundfile
!pip install pytube
!pip install --upgrade pytube
!apt-get install ffmpeg
!pip install pandas gspread oauth2client
!pip install streamlit yt-dlp soundfile numpy matplotlib scipy pandas

⠙⠹⠸⠼
up to date, audited 23 packages in 641ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠼Collecting https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for yt-dlp: filename=yt_dlp-2025.2.19-py3-none-any.whl size=2932679 sha256=9a3f46ea55c10bea2086971a1e1891939fec1060228962d1eef513af473e5954
  Stored in directory: /tmp/pip-ephem-wheel-cache-1a3o0ezc/wheels/2d/79/97/7209650ef73114e0fe0603480da012ad3afacb9cae6b8acd9a
Successfully built yt-dlp
  Attempting uninstall: yt-dlp
    Found existing installation: yt-dlp 2025.2.19
    Uninstalling yt-dlp-2025.2.19:
      Successfully uninstalled yt-dlp-20

Este script en Python implementa una aplicación web interactiva utilizando Streamlit, que permite la descarga, análisis y modulación de señales de audio. Para ello, el código ofrece dos opciones: cargar un archivo de audio local en formato MP3 o WAV, o descargar audio desde YouTube utilizando la biblioteca `yt-dlp`. Una vez obtenido el audio, la señal se visualiza y analiza en el dominio del tiempo y la frecuencia, mostrando su forma de onda y su espectro mediante la transformada de Fourier.

Posteriormente, se implementa un proceso de modulación de amplitud (AM), donde la señal original se multiplica por una portadora senoidal cuya frecuencia puede ajustarse mediante un control en la interfaz. Se permite la reproducción de la señal original, la señal modulada y la señal demodulada después de ser filtrada con un filtro paso bajo Butterworth, diseñado utilizando la biblioteca `scipy.signal`.

El código también emplea `soundfile` para la lectura y escritura de archivos de audio, `numpy` y `matplotlib` para el procesamiento y la graficación de las señales, y Streamlit para la creación de una interfaz interactiva. Esta aplicación es útil para visualizar los efectos de la modulación en señales de audio y comprender mejor los procesos involucrados en las telecomunicaciones.


In [14]:
%%writefile Punto2.py
import os
import yt_dlp as youtube_dl
import subprocess
import soundfile as sf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st

# Título de la aplicación
st.title('Análisis y Modulación de Audio desde YouTube: Descarga, Conversión y Análisis en Streamlit')

# Descripción o texto en la app
st.write("Punto 3:")

# Función para descargar mp3 desde YouTube
def download_ytvid_as_mp3(video_url, name):
    video_info = youtube_dl.YoutubeDL().extract_info(url=video_url, download=False)
    filename = f"{name}.mp3"
    options = {
        'format': 'bestaudio/best',
        'keepvideo': False,
        'outtmpl': filename,
    }
    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])
    return filename

# Subir archivo Excel con enlaces a videos de YouTube
uploaded_file = st.file_uploader("Sube un archivo Excel con enlaces de YouTube", type="xlsx")

if uploaded_file:
    X = pd.read_excel(uploaded_file)
    st.write("Primeras filas del archivo:")
    st.dataframe(X.head())

    # Crear carpeta para resultados
    os.makedirs("results", exist_ok=True)

    # Descargar y procesar el primer video
    n = 0  # Usamos el primer video para el ejemplo
    name_ = f'results/{X.loc[n, "band"]}_{n}_{X.loc[n, "type_num"]}'
    st.write(f"Descargando video: {X.loc[n, 'band']}")

    mp3_file = download_ytvid_as_mp3(X.loc[n, 'link'], name_)
    st.write("Conversión a WAV...")
    subprocess.call(['ffmpeg', '-y', '-i', mp3_file, f'{name_}.wav'])

    # Cargar archivo WAV y mostrar algunos datos
    wav_file = f"{name_}.wav"
    x, fs = sf.read(wav_file)
    st.write(f"Frecuencia de muestreo: {fs} Hz")
    st.audio(wav_file)

    # Partición del archivo de audio
    ts = 5  # Duración del segmento en segundos
    t_segment = np.arange(0, ts, 1/fs)
    x_t = x[:int(ts*fs), :]  # Primer segmento de audio

    st.write("Reproduciendo segmento de audio:")
    st.audio(x_t.T, sample_rate=fs)

    # Generación y visualización de la señal de portadora y mensaje
    xm = x_t[:, 0]  # Canal 1
    Fc = 15000  # Frecuencia de portadora
    Ac = max(abs(xm))  # Amplitud de la portadora
    c = Ac * np.sin(2 * np.pi * Fc * t_segment)

    # Graficar portadora y mensaje
    fig, ax = plt.subplots()
    ax.plot(t_segment, c, label='Portadora')
    ax.plot(t_segment, xm, label='Mensaje')
    ax.set_xlabel('Tiempo [s]')
    ax.set_ylabel('Amplitud')
    ax.legend()
    st.pyplot(fig)

    # Modulación AM
    y = (1 + xm / Ac) * c
    st.write("Reproduciendo señal modulada en AM:")
    st.audio(y, sample_rate=fs)

    # Espectros de las señales
    Xfc = np.fft.fft(c)
    Xfm = np.fft.fft(xm)
    Xfy = np.fft.fft(y)
    vfre = np.fft.fftfreq(len(c), 1/fs)

    # Graficar espectros
    fig, ax = plt.subplots()
    ax.plot(vfre, np.abs(Xfy), label='Modulada')
    ax.plot(vfre, np.abs(Xfm), label='Mensaje')
    ax.plot(vfre, np.abs(Xfc), label='Portadora')
    ax.set_xlim([0, 30000])
    ax.set_xlabel('Frecuencia [Hz]')
    ax.set_ylabel('Magnitud')
    ax.legend()
    st.pyplot(fig)

Overwriting Punto2.py


El código token="2lmAC6DGxB9xF06bq6Js7K7IZQK_5T6aE4rBZBp6hDqefow8M" asigna un valor de cadena (string) a la variable token. Este token se utiliza para autenticar y autorizar la creación de la aplicación en Streamlit, permitiendo el acceso a servicios como ngrok.

In [15]:
token="2mraVkMegLT3dtlJmEyHhXBq8cj_4nGk3GjYEZVfV5XAJR4be"

Este código permite ejecutar una aplicación de Streamlit y hacerla accesible desde Internet utilizando ngrok. Primero, se autentica con ngrok mediante un token de usuario, lo que permite gestionar túneles de forma segura. Luego, se ejecuta el archivo Punto2.py en un servidor local de Streamlit en el puerto 5029, usando nohup para que siga corriendo en segundo plano. Después, se establece un túnel con ngrok, permitiendo exponer el puerto de la aplicación a Internet mediante una URL pública. Finalmente, el código imprime el enlace generado por ngrok, que se puede utilizar para acceder a la aplicación desde cualquier navegador. Esta técnica es útil para compartir dashboards interactivos sin necesidad de abrir puertos manualmente o configurar servidores en la nube.

In [16]:
from pyngrok import ngrok

# # Set authentication token (unique per user)
ngrok.set_auth_token(token)

# # Start Streamlit server on a specific port
!nohup streamlit run Punto2.py --server.port 5029 &

# # Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5029', proto='http', bind_tls=True)

# # Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://ea20-34-13-197-176.ngrok-free.app


# PUNTO 3

Este fragmento de código instala las librerías necesarias para ejecutar una aplicación en Streamlit y facilitar su acceso remoto.

- !pip install streamlit -q: Instala Streamlit, un framework en Python para crear aplicaciones web interactivas, con el argumento -q para que la instalación sea silenciosa.

- !npm install localtunnel: Instala localtunnel, una herramienta que permite exponer servidores locales a Internet sin necesidad de configurar redes o abrir puertos.

- !python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz: Descarga e instala yt-dlp directamente desde su repositorio de GitHub, asegurando una versión actualizada mediante --force-reinstall. Esta biblioteca permite descargar contenido multimedia de plataformas como YouTube.

- !pip install pyngrok: Instala pyngrok, una interfaz en Python para ngrok, que
permite crear túneles seguros para exponer servidores locales a Internet, facilitando el acceso remoto a la aplicación de Streamlit.

In [5]:
!pip install streamlit -q #instalación de librerías
!npm install localtunnel
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install pyngrok

⠙⠹⠸⠼
up to date, audited 23 packages in 655ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠼Collecting https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz


  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
Exception ignored in: <finalize object at 0x7a633241ac40; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.11/weakref.py", line 590, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/urllib3/connectionpool.py", line 1138, in _close_pool_connections
    conn.close()
  File "/usr/lib/python3.11/http/client.py", line 981, in close
    sock.close()   # close it manually... there may be other refs
    ^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 503, in close
    self._real_close()
  File "/usr/lib/python3.11/ssl.py", line 1373, in _real_close
    super()._real_close()
  File "/usr/lib/python3.11/socket.py", line 497, in _real_close
    _ss.close(self)
KeyboardInterrupt: 


Este código implementa una aplicación interactiva en Streamlit para analizar el Total Harmonic Distortion (THD) en rectificadores de onda completa con cargas resistivas y RC. Para ello, genera una señal rectificada de onda completa y calcula su Transformada Rápida de Fourier (FFT) con el fin de analizar su contenido espectral. A partir de la FFT, se calcula el THD, que mide la distorsión armónica total de la señal considerando la relación entre los armónicos superiores y el primer armónico.

La aplicación permite al usuario ajustar la frecuencia de entrada de la señal, así como los valores de la resistencia (R) y la capacitancia (C) de la carga mediante deslizadores interactivos. Posteriormente, se muestran gráficos de la señal en el dominio del tiempo, su espectro de frecuencia (FFT) y los valores de THD obtenidos para cada tipo de carga. Este análisis es especialmente útil en el estudio de circuitos eléctricos y electrónicos, permitiendo evaluar el impacto de diferentes cargas en la distorsión de la señal de salida en rectificadores.

In [6]:
%%writefile Punto3.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq

# Título de la aplicación
st.title('Análisis del Total Harmonic Distortion (THD) en Rectificadores de Onda Completa con Cargas Resistivas y RC')

# Parámetros del sistema
sampling_rate = 10000  # Frecuencia de muestreo
T = 1.0 / sampling_rate  # Intervalo de tiempo
t = np.linspace(0, 1.0, sampling_rate)

# Función para generar la señal del rectificador de onda completa
def rectificador_onda_completa(t, f, R, C=None):
    signal = np.abs(np.sin(2 * np.pi * f * t))
    if C is not None and C > 0:
        tau = R * C
        signal = signal * np.exp(-t / tau)
    return signal

# Función para calcular la FFT
def calcular_fft(signal, sampling_rate):
    N = len(signal)
    yf = fft(signal)
    xf = fftfreq(N, 1 / sampling_rate)[:N // 2]
    return xf, np.abs(yf[:N // 2])

# Función para calcular el THD
def calcular_thd(yf):
    if len(yf) < 2:
        return 0
    V1 = yf[1]  # Primer armónico
    harmonics = np.sqrt(np.sum(yf[2:]**2))
    thd = harmonics / V1 if V1 > 0 else 0
    return thd

# Interfaz gráfica
st.subheader("Parámetros de la simulación")
f = st.slider("Frecuencia de la señal de entrada (Hz):", min_value=10, max_value=100, value=60)
R = st.slider("Ingrese el valor de la Resistencia R (ohmios):", min_value=0.1, max_value=100.0, value=10.0, step=0.1)
C = st.slider("Ingrese el valor de la Capacitancia C (faradios):", min_value=0.0, max_value=10.0, value=1.0, step=0.1)

# Simulación
signal_resistiva = rectificador_onda_completa(t, f, R)
signal_rc = rectificador_onda_completa(t, f, R, C)

# FFT
xf_res, yf_res = calcular_fft(signal_resistiva, sampling_rate)
xf_rc, yf_rc = calcular_fft(signal_rc, sampling_rate)

# Calcular THD
thd_resistiva = calcular_thd(yf_res)
thd_rc = calcular_thd(yf_rc)

# Mostrar resultados de THD
st.write(f"THD para carga resistiva pura: {thd_resistiva:.4f}")
st.write(f"THD para carga RC: {thd_rc:.4f}")

# Graficar señales y FFT
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

axs[0, 0].plot(t[:500], signal_resistiva[:500])
axs[0, 0].set_title("Señal con carga resistiva pura")
axs[0, 0].set_xlabel("Tiempo [s]")
axs[0, 0].set_ylabel("Amplitud")

axs[0, 1].plot(t[:500], signal_rc[:500])
axs[0, 1].set_title("Señal con carga RC")
axs[0, 1].set_xlabel("Tiempo [s]")
axs[0, 1].set_ylabel("Amplitud")

axs[1, 0].plot(xf_res, yf_res)
axs[1, 0].set_title("FFT - Carga resistiva pura")
axs[1, 0].set_xlabel("Frecuencia [Hz]")
axs[1, 0].set_ylabel("Amplitud")

axs[1, 1].plot(xf_rc, yf_rc)
axs[1, 1].set_title("FFT - Carga RC")
axs[1, 1].set_xlabel("Frecuencia [Hz]")
axs[1, 1].set_ylabel("Amplitud")

plt.tight_layout()
st.pyplot(fig)


Overwriting Punto3.py


El código token="2lmAC6DGxB9xF06bq6Js7K7IZQK_5T6aE4rBZBp6hDqefow8M" asigna un valor de cadena (string) a la variable token. Este token se utiliza para autenticar y autorizar la creación de la aplicación en Streamlit, permitiendo el acceso a servicios como ngrok.

In [7]:
token="2mraVkMegLT3dtlJmEyHhXBq8cj_4nGk3GjYEZVfV5XAJR4be"

Este código permite ejecutar una aplicación de Streamlit y hacerla accesible desde Internet utilizando ngrok. Primero, se autentica con ngrok mediante un token de usuario, lo que permite gestionar túneles de forma segura. Luego, se ejecuta el archivo Punto3.py en un servidor local de Streamlit en el puerto 5039, usando nohup para que siga corriendo en segundo plano. Después, se establece un túnel con ngrok, permitiendo exponer el puerto de la aplicación a Internet mediante una URL pública. Finalmente, el código imprime el enlace generado por ngrok, que se puede utilizar para acceder a la aplicación desde cualquier navegador. Esta técnica es útil para compartir dashboards interactivos sin necesidad de abrir puertos manualmente o configurar servidores en la nube.

In [8]:
from pyngrok import ngrok

# # Set authentication token (unique per user)
ngrok.set_auth_token(token)

# # Start Streamlit server on a specific port
!nohup streamlit run Punto3.py --server.port 5039 &

# # Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5039', proto='http', bind_tls=True)

# # Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://3776-34-13-197-176.ngrok-free.app


#PUNTO 4

In [9]:
!pip install streamlit sounddevice numpy scipy matplotlib
!pip install --upgrade pip setuptools wheel
!apt-get install portaudio19-dev
!pip install sounddevice
!pip install pyngrok
#Instalamos laas librerias
!pip install streamlit -q
!pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for yt-dlp: filename=yt_dlp-2025.2.19-py3-none-any.whl size=2932679 sha256=9a3f46ea55c10bea2086971a1e1891939fec1060228962d1eef513af473e5954
  Stored in directory: /tmp/pip-ephem-wheel-cache-bmpm_arg/wheels/2d/79/97/7209650ef73114e0fe0603480da012ad3afacb9cae6b8acd9a
Successfully built yt-dlp
  Attempting uninstall: yt-dlp
    Found existing installation: yt-dlp 2025.2.19
    Uninstalling yt-dlp-2025.2.19:
      Successfully uninstalled yt-dlp-2025.2.19
  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
  Installing build dependencies 

In [10]:
%%writefile Punto4.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.io.wavfile import write
import io
from scipy.io.wavfile import write
import tempfile

# Configuración de página
st.set_page_config(page_title="Detector de Voces", page_icon="📊")
st.title("Detector de voces")
import os
import zipfile
import streamlit as st

def save_uploaded_file(uploaded_file, save_path):
    with open(save_path, "wb") as f:
        f.write(uploaded_file.getbuffer())

def main():
    st.title("Subir y Extraer Archivo ZIP")

    uploaded_file = st.file_uploader("Selecciona un archivo ZIP", type=["zip"])

    if uploaded_file is not None:
        os.makedirs("results", exist_ok=True)
        zip_path = os.path.join("results", uploaded_file.name)
        save_uploaded_file(uploaded_file, zip_path)

        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall("results/")

        st.success(f"Archivos extraídos en 'results/'")

        extracted_files = os.listdir("results/")
        st.write("Archivos extraídos:")
        st.write(extracted_files)

if __name__ == "__main__":
    main()

import os
import pandas as pd
import streamlit as st

def load_audio_data(base_path):
    data = []
    if not os.path.exists(base_path):
        st.error(f"La ruta '{base_path}' no existe.")
        return pd.DataFrame(columns=["NOMBRE DEL ARCHIVO", "ESTUDIANTE", "CLAVE"])

    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith(".wav"):
                    clave = 1 if folder == "correcto" else 2
                    data.append([file, folder, clave])

    return pd.DataFrame(data, columns=["NOMBRE DEL ARCHIVO", "ESTUDIANTE", "CLAVE"])

def main():
    st.title("Explorador de Audios")
    base_path = "results/audios/"
    df = load_audio_data(base_path)

    if not df.empty:
        st.dataframe(df)
    else:
        st.warning("No se encontraron archivos de audio.")

if __name__ == "__main__":
    main()

import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import streamlit as st

def load_and_display_audio(base_path, carpeta_audio, nombre_audio):
    audio_path = os.path.join(base_path, carpeta_audio, nombre_audio)

    if not os.path.exists(audio_path):
        st.error(f"El archivo {nombre_audio} no existe en {carpeta_audio}.")
        return

    y, sr = librosa.load(audio_path, sr=None)

    st.audio(audio_path, format='audio/wav', start_time=0)

    fig, ax = plt.subplots(figsize=(10, 4))
    librosa.display.waveshow(y, sr=sr, ax=ax)
    ax.set_title(f"Forma de onda de {nombre_audio}")
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Amplitud")

    st.pyplot(fig)

    st.write(f"**Shape del audio:** {y.shape}")
    st.write(f"**Frecuencia de muestreo:** {sr} Hz")

def main():
    st.title("Visualización de Audio")

    base_path = "results/audios"
    carpeta_audio = st.text_input("Nombre de la carpeta del audio:", "correcto")
    nombre_audio = st.text_input("Nombre del archivo de audio:", "segment_49.wav")

    if st.button("Cargar y visualizar audio"):
        load_and_display_audio(base_path, carpeta_audio, nombre_audio)

if __name__ == "__main__":
    main()
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st

# Configuración de Streamlit
st.title("Cálculo de la Transformada de Fourier")
st.markdown("Este análisis calcula la Transformada Rápida de Fourier (FFT) de señales de audio y encuentra la frecuencia máxima en cada una.")

# Parámetros simulados (debes reemplazarlos con tus datos reales)
ts = 5  # Duración total en segundos
fs = 48000  # Frecuencia de muestreo en Hz
Ns = 3  # Número de audios simulados

t = np.linspace(0, ts, ts * fs)
x_t = np.random.randn(Ns, len(t))  # Simulación de señales

# Cálculo de la FFT
vf = np.fft.rfftfreq(x_t.shape[1], 1/fs)  # Vector de frecuencias
Xw = np.fft.rfft(x_t, axis=1)  # Transformada de Fourier
Xw_mag = np.abs(Xw)  # Magnitud del espectro

# Encontrar frecuencia máxima por cada audio
idx_max_freq = np.argmax(Xw_mag, axis=1)
f_max = vf[idx_max_freq]

# Mostrar las frecuencias máximas en Streamlit
st.write("### Frecuencia máxima por audio")
for i, freq in enumerate(f_max):
    st.write(f"Audio {i + 1}: {freq:.2f} Hz")

# Graficar formas de onda
st.write("### Forma de onda de todos los audios")
fig, ax = plt.subplots(figsize=(10, 4))
for i in range(Ns):
    ax.plot(np.linspace(0, ts, x_t.shape[1]), x_t[i], alpha=0.5)
ax.set_xlabel("Tiempo [s]")
ax.set_ylabel("Amplitud")
ax.set_title("Forma de onda de todos los audios")
ax.grid(True)
st.pyplot(fig)
# Gráfica del espectro de frecuencia
fig, ax = plt.subplots()
ax.plot(vf, abs(Xw).T)
ax.set_xlabel('f[Hz]')
ax.set_ylabel('|X(f)|')
st.pyplot(fig)

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Se normalizan espectros entre 0 y 1 para evitar inconsistencias por amplitudes máximas
sca = MinMaxScaler()
Xw_ = sca.fit_transform(abs(Xw).T).T

st.title("Espectro de Frecuencia Normalizado")
fig, ax = plt.subplots()
ax.plot(vf, Xw_.T)
ax.set_xlabel('Frecuencia [Hz]')
ax.set_ylabel('|X(f)|')
ax.grid()
st.pyplot(fig)

# En dB
st.title("Espectro de Frecuencia en dB")
fig, ax = plt.subplots()
ax.plot(vf, (20 * np.log10(Xw_ + 1e-10)).T)  # Se suma 1e-10 para evitar discontinuidad del log
ax.set_xlabel('Frecuencia [Hz]')
ax.set_ylabel('|X(f)| dB')
ax.grid()
st.pyplot(fig)

import streamlit as st
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import numpy as np

st.title("Visualización de Datos con t-SNE o PCA")

# Parámetros
perplexity = st.slider("Perplexity", min_value=5, max_value=50, value=20, step=5)
n_components = 2
random_state = 123
learning_rate = "auto"
init_method = "pca"
fmax = 1603

# Selección de reducción de dimensionalidad
method = st.selectbox("Método de reducción de dimensionalidad", ["t-SNE", "PCA"])

if method == "t-SNE":
    reducer = TSNE(perplexity=perplexity, n_components=n_components, random_state=random_state,
                   learning_rate=learning_rate, init=init_method)
elif method == "PCA":
    reducer = PCA(n_components=n_components, random_state=random_state)

# Generar datos de ejemplo
np.random.seed(123)
Xw_ = np.random.rand(100, fmax)  # Datos simulados para ejemplo
label = np.random.randint(0, 2, size=100)  # Etiquetas simuladas
name_c = [f"audio_{i}.wav" for i in range(100)]

# Reducción de dimensionalidad
X_2D = reducer.fit_transform(Xw_[:, :fmax])

# Graficar
fig, ax = plt.subplots(figsize=(8, 6))
scatter = ax.scatter(X_2D[:, 0], X_2D[:, 1], c=label, s=10, cmap="coolwarm")
plt.colorbar(scatter, ax=ax, label="Categoría")

# Etiquetas
color_ = ["b", "y"]
for i, tex in enumerate(name_c):
    nombre = tex[:-4]  # Quita ".wav"
    estudiante = "correcto" if label[i] == 1 else "incorrecto"
    ax.text(X_2D[i, 0] * 1.025, X_2D[i, 1] * 1.025, f"{estudiante}_{nombre}",
            fontsize=6, color=color_[int(label[i])])

st.pyplot(fig)
# Mostrar cantidad de puntos
num_puntos = X_2D.shape[0]
st.write(f"Total de puntos en la gráfica: {num_puntos}")
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler

# Simulación de datos (Reemplazar con los datos reales que ya tienes en tu código)
Xw_ = np.random.rand(100, 1603)  # Datos de espectro normalizados (Ejemplo)
label = np.random.choice([1, 2], 100)  # Etiquetas de ejemplo (1: Correcto, 2: Incorrecto)
name_c = [f"audio_{i}.wav" for i in range(100)]

# Reducción de dimensionalidad
tsne = TSNE(perplexity=20, n_components=2, random_state=123, learning_rate='auto', init='pca')
X_2D = tsne.fit_transform(Xw_[:, :1603])

# Número de puntos en la gráfica
num_puntos = X_2D.shape[0]
st.write(f"Total de puntos en la gráfica: {num_puntos}")

# Gráfico de dispersión con etiquetas
fig, ax = plt.subplots()
color_ = ["b", "y"]  # Azul para correcto, Amarillo para incorrecto
ax.scatter(X_2D[:, 0], X_2D[:, 1], c=label, s=10, cmap='viridis')
ax.set_xlabel("Componente 1")
ax.set_ylabel("Componente 2")
ax.set_title("Visualización de audios en 2D")

for i, tex in enumerate(name_c):
    nombre = tex[:-4]  # Quita la extensión ".wav"
    estudiante = "correcto" if label[i] == 1 else "incorrecto"
    ax.text(X_2D[i, 0] * 1.025, X_2D[i, 1] * 1.025, f"{estudiante}_{nombre}", fontsize=6, color=color_[label[i]-1])

st.pyplot(fig)

Overwriting Punto4.py


In [11]:
token="2lmAC6DGxB9xF06bq6Js7K7IZQK_5T6aE4rBZBp6hDqefow8M"

In [12]:
from pyngrok import ngrok

# # Set authentication token (unique per user)
ngrok.set_auth_token(token)

# # Start Streamlit server on a specific port
!nohup streamlit run Punto4.py --server.port 5049 &

# # Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5049', proto='http', bind_tls=True)

# # Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://60cf-34-13-197-176.ngrok-free.app
